In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [49]:
import pandas as pd
import numpy as np
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
cols = df.columns

In [4]:
df["koi_disposition"].unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [5]:
data_valid = df[df.koi_disposition != 'CANDIDATE']

In [6]:
data_unknown = df[df.koi_disposition == 'CANDIDATE']

In [7]:
data_valid

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983,FALSE POSITIVE,0,1,0,0,21.513523,2.714000e-04,-2.714000e-04,132.335600,0.012200,...,-141,3.508,0.187,-0.153,3.318,0.665,-0.813,287.46786,37.966640,10.630
6986,FALSE POSITIVE,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,...,-152,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
6987,FALSE POSITIVE,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,...,-166,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
6989,FALSE POSITIVE,0,0,1,0,0.681402,2.430000e-06,-2.430000e-06,132.181750,0.002850,...,-236,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [8]:
data_unknown

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
29,CANDIDATE,0,0,0,0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263
47,CANDIDATE,0,0,0,0,40.419504,1.139000e-04,-1.139000e-04,173.564690,0.002220,...,-163,4.507,0.116,-0.105,0.781,0.116,-0.095,294.31686,50.080231,15.487
50,CANDIDATE,0,0,0,0,7.240661,1.620000e-05,-1.620000e-05,137.755450,0.002000,...,-79,4.595,0.012,-0.064,0.765,0.055,-0.028,293.83331,50.230350,15.334
51,CANDIDATE,0,0,0,0,3.435916,4.730000e-05,-4.730000e-05,132.662400,0.011000,...,-115,4.339,0.132,-0.108,1.087,0.157,-0.142,287.88733,46.276241,12.791
59,CANDIDATE,0,0,0,0,1.626630,1.020000e-06,-1.020000e-06,169.820171,0.000487,...,-172,4.410,0.124,-0.186,0.973,0.270,-0.145,294.36819,38.310280,15.279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6979,CANDIDATE,0,0,0,0,4.736816,1.470000e-04,-1.470000e-04,131.787600,0.025600,...,-201,4.456,0.056,-0.224,1.011,0.329,-0.110,289.20331,44.505138,13.922
6980,CANDIDATE,0,0,0,0,130.235324,3.030000e-03,-3.030000e-03,218.271900,0.020100,...,-183,4.529,0.036,-0.192,0.903,0.251,-0.084,289.57452,44.519939,15.991
6984,CANDIDATE,0,0,0,0,8.870416,9.020000e-06,-9.020000e-06,137.481093,0.000869,...,-181,4.027,0.434,-0.186,1.514,0.426,-0.640,290.14914,50.239178,13.579
6985,CANDIDATE,0,0,0,0,47.109631,1.940000e-04,-1.940000e-04,144.131720,0.003430,...,-159,3.597,0.968,-0.242,2.780,1.089,-2.022,296.15601,44.920090,13.731


# Select your features (columns)

In [9]:
# Set features. This will also be used as your x values.
selected_features = data_valid[['koi_period', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad','koi_teq', 'koi_slogg','koi_srad' ]]

In [10]:
result = np.array(data_valid[['koi_disposition']])

In [11]:
selected_features

,koi_period,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_slogg,koi_srad
0,54.418383,0.586,4.50700,874.8,2.83,443,4.467,0.927
1,19.899140,0.969,1.78220,10829.0,14.60,638,4.544,0.868
2,1.736952,1.276,2.40641,8079.2,33.46,1395,4.564,0.791
3,2.525592,0.701,1.65450,603.3,2.75,1406,4.438,1.046
4,4.134435,0.762,3.14020,686.0,2.77,1160,4.486,0.972
...,...,...,...,...,...,...,...,...
6983,21.513523,2.441,77.98300,1583.8,534.47,1006,3.508,3.318
6986,8.589871,0.765,4.80600,87.7,1.11,929,4.296,1.088
6987,0.527699,1.252,3.22210,1579.2,29.35,2088,4.529,0.903
6989,0.681402,0.147,0.86500,103.6,1.07,2218,4.447,1.041


# Create a Train Test Split

Use `koi_disposition` for the y values

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
y = data_valid[['koi_disposition']]
x = data_valid.drop('koi_disposition',axis = 1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.33, random_state=42)

In [15]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
2997,0,0,0,1,1.540389,1.500000e-05,-1.500000e-05,132.004940,0.005800,-0.005800,...,-179,4.422,0.101,-0.188,0.989,0.281,-0.152,285.47534,38.099361,14.753
6216,0,1,1,0,1.075577,7.800000e-08,-7.800000e-08,131.558551,0.000062,-0.000062,...,-210,4.217,0.246,-0.184,1.267,0.342,-0.342,297.91519,43.256203,13.893
1398,0,0,0,0,34.060768,1.372000e-04,-1.372000e-04,141.359280,0.003370,-0.003370,...,-193,4.505,0.052,-0.208,0.940,0.297,-0.099,293.23022,48.281368,15.375
1035,0,0,0,0,16.923421,5.950000e-05,-5.950000e-05,145.814070,0.002800,-0.002800,...,-86,4.465,0.038,-0.120,0.990,0.147,-0.063,294.64816,46.392361,15.818
4605,0,1,0,0,1.213747,2.200000e-08,-2.200000e-08,131.742310,0.000015,-0.000015,...,-260,4.137,0.185,-0.185,1.619,0.492,-0.403,283.86630,47.228271,10.811


In [16]:
y_train.head

<bound method NDFrame.head of      koi_disposition
2997  FALSE POSITIVE
6216  FALSE POSITIVE
1398       CONFIRMED
1035       CONFIRMED
4605  FALSE POSITIVE
...              ...
4043  FALSE POSITIVE
4985  FALSE POSITIVE
6836  FALSE POSITIVE
6880  FALSE POSITIVE
996        CONFIRMED

[3553 rows x 1 columns]>

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [17]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [18]:
X_train_scaled.shape,y_train.shape

((3553, 40), (3553, 1))

In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train.values.ravel())

LabelEncoder()

In [22]:
y_train_encoded = label_encoder.transform(y_train.values.ravel())
y_test_encoded = label_encoder.transform(y_test.values.ravel())

In [23]:
y_train_encoded

array([1, 1, 0, ..., 1, 1, 0])

In [26]:
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [28]:
y_train_categorical

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

# Train the Model



In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=40))
model.add(Dense(units=20, activation='relu', input_dim=40))
model.add(Dense(units=2, activation='softmax'))

In [37]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train_categorical, epochs=100,shuffle = True, verbose = 1)

Epoch 1/100
112/112 [==============================] - 1s 2ms/step - loss: 0.4456 - accuracy: 0.7563
Epoch 2/100
112/112 [==============================] - 0s 1ms/step - loss: 0.1521 - accuracy: 0.9901
Epoch 3/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0528 - accuracy: 0.9921
Epoch 4/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0390 - accuracy: 0.9924
Epoch 5/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0347 - accuracy: 0.9921
Epoch 6/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0336 - accuracy: 0.9924
Epoch 7/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0322 - accuracy: 0.9924
Epoch 8/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0314 - accuracy: 0.9924
Epoch 9/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0315 - accuracy: 0.9921
Epoch 10/100
112/112 [==============================] - 0s 2ms/step - loss: 0.0299 - accura

112/112 [==============================] - 0s 2ms/step - loss: 0.0189 - accuracy: 0.9955
Epoch 82/100
112/112 [==============================] - 0s 2ms/step - loss: 0.0215 - accuracy: 0.9947
Epoch 83/100
112/112 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.9947
Epoch 84/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0170 - accuracy: 0.9947
Epoch 85/100
112/112 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 0.9952
Epoch 86/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0170 - accuracy: 0.9952
Epoch 87/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0168 - accuracy: 0.9961
Epoch 88/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0170 - accuracy: 0.9961
Epoch 89/100
112/112 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 0.9949
Epoch 90/100
112/112 [==============================] - 0s 1ms/step - loss: 0.0164 - accuracy: 

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_7 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 42        
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


In [39]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

112/112 [==============================] - 0s 1ms/step - loss: 0.0139 - accuracy: 0.9961
Training Data Score: [0.013903025537729263, 0.9960596561431885]
55/55 [==============================] - 0s 1ms/step - loss: 0.0566 - accuracy: 0.9863
Testing Data Score: [0.05660337582230568, 0.9862935543060303]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [45]:
# # Create the GridSearchCV model
# from sklearn.model_selection import GridSearchCV
# param_grid = dict(epochs=[10,20,30])
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring="accuracy")


In [47]:

# grid_result = grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [52]:
filename = 'Geoffrey_model_1.h5'
model.save( filename)